# Simplify Racah expressions by means of sum rules

In [1]:
using JAC, SymEngine

**Note**: The Julia package `SymEngine` is needed to perform symbolic simplifications of Racah algebra expressions in JAC but, by default, is not automatically loaded.

As mentioned before, the (data) type `RacahExpression` is very central to applying the techniques from Racah's algebra. This data type enables one to *comprise* -- less or more -- sophisticated expressions into a single Julia variable and to attempt its simplification by a set of internal (sum) rules:

In [2]:
? RacahExpression

search: RacahExpression



`struct  RacahAlgebra.RacahExpression`  ... defines a type for a RacahExpression with symbolic arguments.

```
+ summations    ::Array{Basic,1}      ... Summation indices.
+ integrals     ::Array{Integral,1}   ... Integrals.
+ phase         ::Basic               ... Phase of the Racah expression.
+ weight        ::Basic               ... Weight of the Racah expression.
+ deltas        ::Array{Kronecker,1}  ... List of Kronecker deltas.
+ triangles     ::Array{Triangle,1}   ... List of Triangle deltas.
+ w3js          ::Array{W3j,1}        ... List of Wigner 3-j symbols
+ w6js          ::Array{W6j,1}        ... List of Wigner 6-j symbols
+ w9js          ::Array{W9j,1}        ... List of Wigner 9-j symbols
+ ylms          ::Array{Ylm,1}        ... List of spherical harmonics
+ djpqs         ::Array{Djpq,1}       ... List of (small) Wigner rotation matrices
```

---

`RacahAlgebra.RacahExpression()`       ... constructor for defining an empty RacahExpression.

---

`RacahAlgebra.RacahExpression(rex::RacahAlgebra.RacahExpression;`

```
    summations=..,      integrals=..,   phase=..,       weight=..,      deltas=..,     triangles=..,       
    w3js=..,            w6js=..,        w9js=..,        ylms=..,       djpqs=..) 
                
... constructor for modifying a given rex::RacahExpression by 'overwriting' the explicitly selected parts of the 
    expression.
```


As seen from the definition of this (data) struct, the different 'delta' and Wigner *symbols* of such an expression are kept and maintained separately, so that the known *sum rules* can be applied more readily. Moreover, the last constructor of a `RacahExpression` shows that it quite simple to overwrite or extent an already *existing* `RacahExpression`, starting from a simple $\mathbb{1}$:

In [3]:
RacahExpression()

1

Perhaps, the *simplest* sum rules refer to the orthogonality of the Wigner 3j and 6j symbols; for example, the Wigner 6j symbols fullfill the following **orthogonality relations** which can be displayed (just for illustration here) by:

In [4]:
? RacahAlgebra.sumRulesForTwoW6j

`RacahAlgebra.sumRulesForTwoW6j(rex::RacahAlgebra.RacahExpression)`       ... attempts to find a simplification of the given Racah expression by using sum rules for one Wigner 6-j symbol.          Once a simplification is found, no attempt is made to find another simplifcation for this set of rules.         A (istrue, rex)::Tuple{Bool, RacahExpression} is returned but where rex has no meaning for !istrue.

```
                                             2
                                  ( X  Y  Z )
            Sum(X,Y,Z)  [X,Y,Z]  {(         )}    =    [a,b,c]
                                  ( a  b  c )
             
             
                         ( a  b  X )   ( c  d  X )        1
            Sum(X) [X]  {(         )} {(         )}  =   ---  d(p,q) d(a,d,p) d(b,c,p)
                         ( c  d  p )   ( a  b  q )       [p]
```


Both of these *rules* show that a summation over the product of two Wigner 6j symbols can be re-written just in terms of some quantum numbers and triangle conditions. Note that $[a,b,...] = (2a+1)\: (2b+1)\: ...$. More general, all the implemented sum rules are displayed as *inline* comments in the code, although not as *docstrings* (apart from this particular function here). 

Typically, *only some standard form of each sum rule is shown in the literature*, and many of these sum rules are just displayed in quite specialized texts about angular momenta. Likely, the most comprehensive compilation of these (and many other) rules can be found in the **monograph by Varshalovich et al. (1988)**. --- In general, however, one needs to recognize all the symmetries of a Racah expressions, implying all the phases and possible (weight) factors that arise from these symmetries. In JAC, this is realized by cycling automatically through all symmetric forms of the Wigner nj (n = 3,6,9) symbols. In a later step, we also plan to take the spherical harmonics and the Wigner rotation matrices into account as well into the internal representation of a `RacahExpression`.

Again, let us first declare some `Basic` variables which we can later apply to define our first `RacahExpression`:

In [5]:
a = Basic(:a);    b = Basic(:b);    c = Basic(:c);    d  = Basic(:d);    ee  = Basic(:ee);    f  = Basic(:f)
g = Basic(:g);    h = Basic(:h);    k = Basic(:k);    l  = Basic(:l);    p   = Basic(:p);     q  = Basic(:q);     
r = Basic(:r);    s = Basic(:s);    t = Basic(:t);    X  = Basic(:X);    Y   = Basic(:Y);     Z  = Basic(:Z);

In [6]:
aw6j = W6j(X, Y, Z, a, b ,c);    bw6j = W6j(X, Y, Z, a, b ,c)
rex  = RacahExpression( [X, Y, Z], Integral[], Basic(0), Basic((2*X+1)*(2*Y+1)*(2*Z+1)), 
                        Kronecker[], Triangle[], W3j[], W6j[aw6j, bw6j], W9j[], Ylm[], Djpq[] )

Sum_[Basic[X, Y, Z]]  ((1 + 2*Y)*(1 + 2*Z)*(1 + 2*X))  W6j{X, Y, Z; a, b, c}  W6j{X, Y, Z; a, b, c}  

As before, we can simply *evaluate* this expression which attempts to apply one of the -- more than 45 implemented -- sum rules in order to *reduce* either the number of Wigner symbols *and/or* the number of summation indices:

In [7]:
RacahAlgebra.evaluate(rex)

>> Apply sum rule for two W6j -- Sum(X,Y,Z) [X,Y,Z] ...


((1 + 2*b)*(1 + 2*c)*(1 + 2*a))  

This example looks perhaps quite *over-simplified* as we could use exactly the *orthogonaly relation* from above to get this result. However, the same simplification also works if we first randomly re-write the given Racah expression and then attempt its simplification again.

In [8]:
rex2 = RacahAlgebra.equivalentForm(rex);   @show rex2
RacahAlgebra.evaluate(rex2)

>> Select 3th equivalent form for W6j{X, Y, Z; a, b, c}    ==>   W6j{Y, X, Z; b, a, c}  
>> Select 13th equivalent form for W6j{X, Y, Z; a, b, c}    ==>   W6j{a, b, Z; X, Y, c}  
rex2 = Sum_[Basic[X, Y, Z]]  ((1 + 2*Y)*(1 + 2*Z)*(1 + 2*X))  W6j{Y, X, Z; b, a, c}  W6j{a, b, Z; X, Y, c}  
>> Apply sum rule for two W6j -- Sum(X) [X] ...


Sum_[Basic[X, Y]]  ((1 + 2*Y)*(1 + 2*X)/(1 + 2*c))  delta (b, X, c)  delta (a, Y, c)  

You may test this simplification several times for (randomly) different *equivalent forms* of `rex` and, likely, will receive slightly different results with regard to the number of symbols and summations. This is related to the **phase issue**, which refers to the fact that it is not easy to always recognize how the overall phase can be re-written internally so that a particular sum rule applies. Here, we note that the application of any sum rule *always requests* that all other parts of the given Racah expression, including its overall phase, must be independent of those parts which are to be removed. Further (formal) improvement on this **phase issue** might be possible but, sometimes, these equivalences need to be recognized and corrected *manually*.

Of course, we can simplify also less obvious Racah expressions, such as:

In [13]:
aw6j = W6j(a, b, X, c, d, p);    bw6j = W6j(c, d, X, b, a, q)
rex  = RacahExpression( [X], Integral[], Basic(X), Basic(2*X+1), Kronecker[], Triangle[], W3j[], W6j[aw6j, bw6j], W9j[], Ylm[], Djpq[] )
@show rex
rex  = RacahAlgebra.equivalentForm(rex)
RacahAlgebra.evaluate(rex)

rex = Sum_[Basic[X]]  (-1)^(X)  (1 + 2*X)  W6j{a, b, X; c, d, p}  W6j{c, d, X; b, a, q}  
>> Select 11th equivalent form for W6j{a, b, X; c, d, p}    ==>   W6j{p, a, d; X, c, b}  
>> Select 4th equivalent form for W6j{c, d, X; b, a, q}    ==>   W6j{d, X, c; a, q, b}  
>> Apply sum rule for two W6j -- Sum(X) (-1)^X [X] (1)^X ...


(-1)^(-p - q)  W6j{c, b, p; d, a, q}  

In [15]:
aw6j = W6j(a, f, X, ee, b, s);   bw9j = W9j(a, f, X, d, q, ee, p, c, b)
rex  = RacahExpression( [X], Integral[], Basic(0), Basic(2*X+1), Kronecker[], Triangle[], W3j[], W6j[aw6j], W9j[bw9j], Ylm[], Djpq[] )
@show rex
rex  = RacahAlgebra.equivalentForm(rex)
RacahAlgebra.evaluate(rex)

rex = Sum_[Basic[X]]  (1 + 2*X)  W6j{a, f, X; ee, b, s}  W9j{a, f, X; d, q, ee; p, c, b}  
>> Select 13th equivalent form for W6j{a, f, X; ee, b, s}    ==>   W6j{ee, b, X; a, f, s}  
>> Select 71th equivalent form for W9j{a, f, X; d, q, ee; p, c, b}    ==>   W9j{b, c, p; ee, q, d; X, f, a}  
>> Apply sum rule for one W6j & one W9j -- Sum(X) [X] (-1)^X ...


(-1)^(2*s)  W6j{d, c, s; f, ee, q}  W6j{d, c, s; b, a, p}  

Apart from these quite simple expressions, much more complex ones rapidly arise if angular momenta are coupled together or *re-coupled* in order to allow the simplification of many-particle matrix elements.

In the next example, we shall consider the (so-called) *re-coupling coefficients* $ <(j_1,j_2)\, J_{12}, j_3; JM| j_1, (j_2,j_3)\,J_{23}; JM>$ which is known to be independent of $M$. The expression of this re-coupling coefficient can be written down quite easily by applying twice a Clesch-Gordan expansion on both sides of the 'overlap matrix element'. Simple manipulations gives immediately rise to the `RacahExpression`:

In [17]:
j1   = Basic(:j1);    j2 = Basic(:j2);    j3 = Basic(:j3);    J12 = Basic(:J12);    J23 = Basic(:J23);    J = Basic(:J)
m1   = Basic(:m1);    m2 = Basic(:m2);    m3 = Basic(:m3);    M12 = Basic(:M12);    M23 = Basic(:M23);    M = Basic(:M)
w3ja = W3j(J12, j3, J, M12, m3, -M);        w3jb = W3j(j1, j2, J12, m1, m2, -M12)       
w3jc = W3j(j2, j3, J23, m2, m3, -M23);      w3jd = W3j(j1, J23, J, m1, M23, -M)   
rex  = RacahExpression( [m1, m2, m3, M12, M23], Integral[], -J12 + 2*j3 - 2*M - 2*j1 - M12 - M23 + J23, 
            (2*J+1) * sqrt( (2*J12+1)*(2*J23+1) ), Kronecker[], Triangle[], [w3ja, w3jb, w3jc, w3jd], W6j[], W9j[], Ylm[], Djpq[] )

Sum_[Basic[m1, m2, m3, M12, M23]]  (-1)^(-J12 + J23 - 2*M - M12 - M23 - 2*j1 + 2*j3)  ((1 + 2*J)*sqrt((1 + 2*J12)*(1 + 2*J23)))  W3j(J12, j3, J; M12, m3, -M)  W3j(j1, j2, J12; m1, m2, -M12)  W3j(j2, j3, J23; m2, m3, -M23)  W3j(j1, J23, J; m1, M23, -M)  

which includes a five-fold summation (three further summations, for instance, for $m1'$, $m2'$ and $m3'$ can be simplified because of the assumed normalization of the $|j_p m_p>$ states of all subsystems). Here, the simplification of this `RacahExpression` is already harder to *see* but can be obtained by calling:

In [12]:
RacahAlgebra.evaluate(rex)

>> Apply sum rule for two W6j -- Sum(X,Y,Z) [X,Y,Z] ...


((1 + 2*b)*(1 + 2*c)*(1 + 2*a))  

The given recoupling coefficient is obviously independent of $M$ and just given by a Wigner 6j symbol times some rather trivial (delta) factors. A closer inspection of the Wigner symbol also enables one to express the phase in a slightly more convinient form: $(-1)^{j_1 + j_2 + j_3 + J}$; cf. the **phase issue**.

## **The following part of this nootebook is still under construction.**